<a href="https://colab.research.google.com/github/Harshdupare/Leaf-segmentation-using-SegNet/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow pandas numpy pillow matplotlib


In [ ]:
import os
dataset_path = "/content/drive/MyDrive/leaf/Leaf"

if os.path.exists(dataset_path):
    print("Directory exists:", os.listdir(dataset_path))
else:
    print("Directory does not exist. Check the path!")

Directory exists: ['test.txt', 'val.txt', 'SegNet_compress2_covid_mini (1).py', 'SegNet_compress2_covid_mini.plot.all.py', 'model_5l_weight_leaf_segnet.weights.h5', '.DS_Store', 'val', 'test-colormap', 'train', 'test', 'train-colormap', 'val-colormap', '.ipynb_checkpoints', 'train.txt']


In [ ]:

# Change directory to your dataset location
import os
dataset_path = "/content/drive/MyDrive/leaf/Leaf/"

import numpy as np
import pandas as pd
import sys
from PIL import Image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization, Input, Convolution2D, MaxPooling2D, UpSampling2D, Activation, Reshape
from tensorflow.keras.models import Model

# Define Constants
img_w, img_h = 256, 256
n_labels = 3
Tree = [128, 128, 0]    #yellowish
leaf = [0, 255, 0]      #Green
Unlabelled = [0, 0, 0]  #black


n_train = 65
n_test = 10
n_val = 30


# Label Mapping Function
'''
this function flips the 0s to 1s according to the labels given, like black , green and yellow
'''
def label_map1(labels):
    label_map = np.zeros([img_h, img_w, n_labels])
    for r in range(img_h):
        for c in range(img_w):
            if np.array_equal(labels[r][c], Unlabelled):
                label_map[r, c, 0] = 1
            elif labels[r][c][1] > 220 and labels[r][c][0] < 25 and labels[r][c][2] < 25:
                label_map[r, c, 1] = 1
            elif 100 < labels[r][c][0] < 160 and 100 < labels[r][c][1] < 160 and labels[r][c][2] < 40:
                label_map[r, c, 2] = 1
    return label_map

# Data Preprocessing
def prep_data1(mode):
    assert mode in {'test', 'train', 'val'}, \
        'mode should be either \'test\' or \'train\''
    data = []
    label = []
    file1 = open(dataset_path + mode + '.txt', 'r')
    files = file1.read().split('.JPG')
    n = n_train if mode == 'train' else n_test

    if(mode == 'val'):
        n = n_val

    idx = 0
    for filename in files:
        print(filename)
        if(filename == ""):
            break
        idx += 1
        if(idx > n):
            break
        new_im = Image.open(dataset_path + mode + '/' + filename + '.JPG')

        new_im1 = Image.open(dataset_path + mode + '-colormap/' + filename + '.JPG')


        #img, gt = [imread(path + mode + '/' + filename + '.JPG')], imread(path + mode + '-colormap/' + filename + '.JPG')

        img, gt = [np.array(new_im,dtype=np.uint8)], np.array(new_im1,dtype=np.uint8)
        data.append(np.reshape(img,(256,256,3)))
        label.append(label_map1(gt))
        sys.stdout.write('\r')
        sys.stdout.flush()
    sys.stdout.write('\r')
    sys.stdout.flush()
    data, label = np.array(data), np.array(label).reshape((n, img_h * img_w, n_labels))

    print( mode + ': OK')
    print( '\tshapes: {}, {}'.format(data.shape, label.shape))
    print( '\ttypes:  {}, {}'.format(data.dtype, label.dtype))
    print( '\tmemory: {}, {} MB'.format(data.nbytes / 1048576, label.nbytes / 1048576))

    return data, label

# Define SegNet Model
def SegNet(input_shape=(256, 256, 3), classes=3):
    img_input = Input(shape=input_shape)
    x = Convolution2D(64, (3, 3), padding="same", activation="relu")(img_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Convolution2D(128, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Convolution2D(256, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Convolution2D(512, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    x = Convolution2D(512, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Convolution2D(256, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Convolution2D(128, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Convolution2D(64, (3, 3), padding="same", activation="relu")(x)
    x = BatchNormalization()(x)

    x = Convolution2D(classes, (1, 1), activation="softmax")(x)
    x = Reshape((input_shape[0] * input_shape[1], classes))(x)

    model = Model(img_input, x)
    return model

# Initialize Model
model = SegNet()
optimizer = SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
model.summary()

# Load Data
train_data, train_label = prep_data1('train')
val_data, val_label = prep_data1('val')

# Train Model
nb_epoch = 60
batch_size = 2
history = model.fit(train_data, train_label, batch_size=batch_size, epochs=nb_epoch, validation_data=(val_data, val_label))

# Save Model Weights
#model.save_weights('/content/drive/MyDrive/model_weights.keras')
model.save('/content/drive/MyDrive/model.keras')


# Test Model
test_data, test_label = prep_data1('test')
score = model.evaluate(test_data, test_label)
print(f"Test Loss: {score[0]}, Test Accuracy: {score[1]}")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 256, 256, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 128, 128, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 64, 64, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 32, 32, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 32, 32, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_3 (UpSampling2D)       │ (None, 64, 64, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 64, 64, 256)         │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 64, 64, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_4 (UpSampling2D)       │ (None, 128, 128, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 5,467,395 (20.86 MB)

 Trainable params: 5,463,555 (20.84 MB)

 Non-trainable params: 3,840 (15.00 KB)

00695906-210d-4a9d-822e-986a17384115___RS_LB 4026
006cc01a-1888-4326-9b62-a65cb55a6377___RS_LB 5207
0114b195-844c-4978-93a9-b0d5aae010f1___RS_LB 2738
01270f5c-a44b-4da7-9398-289088c197ab___RS_LB 2517
0201bf8e-27e3-4c92-ae97-2fb4f665b5d3___RS_LB 4320
02212653-e92b-49aa-96b7-9497af51e502___RS_LB 3151
025f3c76-6d6c-4dad-ba22-e3e72d084ef1___RS_LB 3105
031672c9-bf0a-46bf-8b52-d7cdc067612e___RS_LB 5019
031c39d3-9a91-4e81-85ac-b36512133b0a___RS_LB 5220
034a54f1-e3e9-4524-a318-9d253d9f1394___RS_LB 2893
0396e413-e165-427a-ba72-04cbae7b8ab0___RS_LB 4751
03bab48b-962b-4c08-bc98-da95237d20ac___RS_LB 3190
0441138d-5f9f-4ede-ab9a-49edabc605b3___RS_LB 4235
0450570b-44d1-4290-956d-5d970164a2e2___RS_LB 5160
04ce7b2d-5e41-43c6-9e11-e1bc4f5e4cca___RS_LB 3224
04fe5855-ec9c-40b3-9893-ca8addc236bd___RS_LB 4913
050b6d37-4056-4c6c-8359-d7823a488531___RS_LB 3944
051089cf-e083-4fdb-9256-aac12b8a55d2___RS_LB 4078
0525ac6e-1d1f-4866-89bd-af4b8ba9c46b___RS_LB 4814
053c5330-129d-4515-84da-82a701710723___RS_LB 4576
